# ResNet-34

ResNet, short for Residual Network, is a deep neural network architecture that was introduced in a groundbreaking paper titled "Deep Residual Learning for Image Recognition" by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun in 2015. ResNet is a significant advancement in the field of deep learning, particularly for image recognition tasks, and it addressed the problem of training very **deep neural networks**.


![](./Assets/Degradation.png)

The **"degradation problem"** in deep neural networks refers to the phenomenon where, as you **increase the depth (number of layers) of a neural network, its training accuracy saturates and then starts to degrade (get worse) rather than improve.** In other words, adding more layers to a network should ideally lead to better performance, but in practice, this was not the case. The degradation problem was particularly problematic for very deep neural networks.

The degradation problem occurred due to the difficulty of training deep networks effectively. As networks became deeper, they suffered from **vanishing gradients**, which is a problem during the backpropagation process. Vanishing gradients occur when the gradients of the loss function with respect to the network's parameters become extremely small as they are backpropagated through the many layers of the network. When gradients are too small, it becomes very challenging to update the network's weights effectively, and as a result, training stagnates or even diverges.

**ResNet addressed the degradation problem and enabled the training of very deep networks by introducing the concept of residual learning.**
Here's how ResNet solved the degradation problem:
**Residual Blocks:** Instead of trying to learn the desired mapping directly from the input to the output, ResNet focuses on learning the residual (the difference) between the input and the output of a block. This can be mathematically represented as F(x) = H(x) + x, where:
**'x'** is the input to the block.
**'H(x)'** is the learned residual mapping (the part of the mapping that the network needs to learn).
**'F(x)'** is the desired output.

**Skip Connections:** ResNet uses skip connections (shortcut connections) that allow the identity information from the input (x) to be directly added to the output of the block. These skip connections create paths for the gradient to flow more easily during backpropagation. Essentially, they provide a shortcut for gradients to bypass the layers that may cause vanishing gradients. The key idea is that if the network wants to make no change to the input, it can do so by simply setting H(x) to zero.

**Stable Training:** Because of the skip connections and the residual learning approach, ResNet architectures can be very deep (hundreds of layers) without suffering from the degradation problem. The gradients can flow through the network, even in deep architectures, which makes it possible to train very deep networks effectively.

In summary, ResNet solved the degradation problem in deep neural networks by introducing residual blocks and skip connections, which allowed for the training of extremely deep networks without suffering from vanishing gradients. This innovation was crucial in advancing the field of deep learning, particularly for image recognition tasks, and led to significant improvements in model accuracy and performance.

![](./Assets/Resnet_model.png)

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl
import matplotlib.pyplot as plt
import time

In [2]:
# Loading dataset
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.cifar10.load_data()

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(X_train[0].shape)

170498071/170498071 [==============================] - 101s 1us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)
(32, 32, 3)


In [3]:
figure = plt.figure(figsize=(12,12))
for i in range(12, 21):
    plt.subplot(3,3,i-11)
    plt.xlabel(Y_train[i])
    plt.imshow(X_train[i])
plt.show()

In [4]:
print(X_train.dtype, X_test.dtype)
X_train = X_train.astype(float)
X_test = X_test.astype(float)
X_train = X_train/255.0
X_test = X_test/255.0

uint8 uint8


In [5]:
# Creating a python list containing thr classes for referencing the clas names of the ./Images during visualization
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [6]:
def identity_block(x, filter):

    x_skip = x

    # Layer 1
    x = tfl.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tfl.BatchNormalization(axis=3)(x)
    x = tfl.Activation('relu')(x)

    # Layer 2
    x = tfl.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tfl.BatchNormalization(axis=3)(x)

    # Plain Skip connection
    x = tfl.Add()([x, x_skip])
    x = tfl.Activation('relu')(x)

    return x

In [7]:
def convolutional_block(x, filter):

    x_skip = x

    # Layer 1
    x = tfl.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tfl.BatchNormalization(axis=3)(x)
    x = tfl.Activation('relu')(x)

    # Layer 2
    x = tfl.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tfl.BatchNormalization(axis=3)(x)

    # Skip Connection with Convolution layer
    x_skip = tfl.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    x = tfl.Add()([x, x_skip])
    x = tfl.Activation('relu')(x)

    return x


# Let's implement the model now.
The identity shortcuts can be directly used when the input and output are of the same dimensions (solid line shortcut). When the dimensions increase (dotted line shortcuts), we consider two options:
(A) The shortcut still performs identity mapping, with extra zero entries padded for increasing dimensions. This option introduces no extra
parameter.
(B) The projection shortcut is used to match dimensions **(done by 1×1 convolutions).** 
For both options, when the shortcuts go across feature maps of two sizes, they are performed with a **stride of 2.**
![](./Assets/Resnet34_Architecture.png)

In [8]:
def ResNet34(shape = (32, 32, 3), classes = 10):

    #Stage 1
    x_input = tfl.Input(shape)
    x = tfl.ZeroPadding2D((3, 3))(x_input)

    # Stage 2
    x = tfl.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tfl.BatchNormalization()(x)
    x = tfl.Activation('relu')(x)
    x = tfl.MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    block_layers = [3, 4, 6, 3]
    filter_size = 64

    # Stage 3
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)

    # Stage 4
    x = tfl.AveragePooling2D((2,2), padding = 'same')(x)
    x = tfl.Flatten()(x)
    x = tfl.Dense(512, activation = 'relu')(x)
    x = tfl.Dense(classes, activation = 'softmax')(x)

    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [9]:
model = ResNet34()
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 38, 38, 3)            0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 19, 19, 64)           9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 19, 19, 64)           256       ['conv2d[0][0]']       

In [10]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [15]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 32)

Epoch 1/5
1563/1563 [==============================] - 577s 369ms/step - loss: 0.2825 - accuracy: 0.9024
Epoch 2/5
1563/1563 [==============================] - 600s 384ms/step - loss: 0.2352 - accuracy: 0.9181
Epoch 3/5
1563/1563 [==============================] - 634s 406ms/step - loss: 0.1817 - accuracy: 0.9361
Epoch 4/5
1563/1563 [==============================] - 612s 391ms/step - loss: 0.1553 - accuracy: 0.9463
Epoch 5/5
1563/1563 [==============================] - 598s 382ms/step - loss: 0.1395 - accuracy: 0.9525


In [17]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

313/313 [==============================] - 17s 54ms/step - loss: 1.2151 - accuracy: 0.7356
Loss = 1.2150697708129883
Test Accuracy = 0.7355999946594238
